In [3]:
import random
import bitstring
from bitstring import Bits

# Packet Gen

In [4]:
def packet_gen(largoBytes, semilla = 10):
    packet = bytearray(largoBytes)
    for i in range(0, largoBytes):
        packet[i] = random.getrandbits(8)
    packet = Bits(packet)
    return(packet)

# Tiles Gen + padding

In [5]:
def get_tiles(bitsPacket, tileLengthBits):
    tiles = []
    #verificar si el tamaño de packet es multiplo del tamaño de tile
    if bitsPacket.length%tileLengthBits > 0:
        #no es multiplo -> agregar padding
        padding = tileLengthBits - bitsPacket.length%tileLengthBits
        bitsPacketPadding = bitsPacket + Bits(padding)
    initIndex = 0
    finalIndex = tileLengthBits
    paddedPacketLength = bitsPacketPadding.length
    while finalIndex <= paddedPacketLength:
        tile = bitsPacketPadding[initIndex:finalIndex]
        tiles.append(tile)
        initIndex += tileLengthBits
        finalIndex += tileLengthBits
    return tiles, padding

In [22]:
def packet_padding(packet, bitsMultiple):
    newPacket = Bits(packet)
    if newPacket.length%bitsMultiple > 0:
        #no es multiplo de 1 byte
        padding = bitsMultiple - newPacket.length%bitsMultiple
        newPacket = newPacket + Bits(padding)
    return newPacket, padding

In [25]:
p = Bits(uint=15, length = 6)
np, padding = packet_padding(p,16)
print(p)
print(p+None)



0b001111
0b001111


# Headers gen


In [6]:
def get_regular_header(ruleId, dTag, W, FCN):
    #RuleId field
    rId = Bits(uint=ruleId["ID"], length=8) 
    #DTag field
    if ruleId["PROFILE"]["DTAG_SIZE"] == 0:
        dT = None
    else:
        dT = Bits(uint=dTag, length=ruleId["PROFILE"]["DTAG_SIZE"])
    #W field
    w   = Bits(uint=W, length=8)[len(w)-ruleId["PROFILE"]["W_SIZE"]:len(w)] #toma los M bits menos significativos
    #FCN field
    fcn = Bits(uint=FCN, length=ruleId["PROFILE"]["FCN_SIZE"])
    header = rId + dT + w + fcn
    return header, rId, dT, w, fcn

def get_all1_header(ruleId, packet, dTag, W):
    rId = Bits(uint=ruleId["ID"], length=8) 
    dT  = Bits(uint=dTag        , length=ruleId["PROFILE"]["DTAG_SIZE"])
    w   = Bits(uint=W           , length=8)[len(w)-ruleId["PROFILE"]["W_SIZE"]:len(w)]
    fcn = ~Bits(length=ruleId["PROFILE"]["FCN_SIZE"])

# CRC32 implementation

In [7]:
class CRC32(object):
    '''
    Class who carry the crc32 algorithm implementation
    '''
    def __init__(self):
        self.table = create_table()
    
    def create_table(self):
        table = []
        for i in range(256):
            k = i
            for j in range(8):
                if k & 1:
                    k ^= 0x1db710640
                k >>= 1
            table.append(k)
        return table
        
    def calc(self, buf, crc=0):
        crc ^= 0xffffffff
        for k in buf:
            crc = (crc >> 8) ^ self.table[(crc & 0xff) ^ k]
        return crc ^ 0xffffffff

In [8]:
paquete = packet_gen(largoBytes=24)
tiles, padding = get_tiles(paquete, 7)

# Rules and Profiles Structure

In [9]:
ruleIdElements = [
    "ID", #number
    "FRAGMENTATION", #Si es un tipo fragmentado
    "MODE", #modo de fragmentacion
    "PROFILE" #profile a ocupar
]

profileElements = [
    "FCN_SIZE", #[size en bits]
    "DTAG_SIZE", #[size en bits]
    "W_SIZE", #[size en bits]
    "MIC", #[size, algorithm]
    "WINDOW_SIZE", #[value]
    "MAX_ACK_REQUESTS", #[value]
    "RETRANSMISSION_TIMER", #[value_seconds]
    "INACTIVITY_TIMER" #[value_seconds]
]

# Some rules definition

In [10]:
profile1 = {
    "RULEID_SIZE": 8,
    "DTAG_SIZE": 8, 
    "W_SIZE": 1, 
    "FCN_SIZE": 8,
    "MIC": [32, "CRC32"], 
    "WINDOW_SIZE": 5,
    "MAX_ACK_REQUESTS": 10,
    "RETRANSMISSION_TIMER": 10, 
    "INACTIVITY_TIMER": 10 
}

rule1 = {
    "ID": 1,
    "FRAGMENTATION": True,
    "MODE": "always-ack",
    "PROFILE": profile1
}

rule2 = {
    "ID": 2,
    "FRAGMENTATION": True,
    "MODE": "always-ack",
    "PROFILE": profile1
}

rule3 = {
    "ID": 2,
    "FRAGMENTATION": True,
    "MODE": "always-ack",
    "PROFILE": profile1
}

rules = {
    rule1["ID"]: rule1,
    rule2["ID"]: rule2,
    rule3["ID"]: rule3
}

In [11]:
def send_ack_always(packet, rule, dTag, fragment_size_bytes = 100):
    #asegurarse que el paquete sea de tipo Bits
    packet = Bits(packet)
    #Tamaño de header en Bits
    reg_hdr_size_bits = rule["PROFILE"]["RULEID_SIZE"]+rule["PROFILE"]["DTAG_SIZE"]+rule["PROFILE"]["W_SIZE"]+rule["PROFILE"]["FCN_SIZE"]
    tiles_size_bits = fragment_size_bytes*8 - reg_hdr_size_bits #se calcula tamaño de cada tile
    max_windows_number = math.ceil(packet.length/tiles_size) #se calcula el numero total de ventanas a transmitir
    
    

In [131]:
bits(paquete)

Bits('0x1ce0fee655ddf567d7996d426fc8ebc6c22850da66dc9c2c')

In [32]:
def get_window(tiles, windowSize, windowNumber):
    window = []
    tilesNumber = len(tiles)
    index = windowNumber*windowSize
    while(index < tilesNumber):
        window.append(tiles[index])
        index = index+1
    return window